To get more efficient algorithm for solving the game we need to better identify when a ship has been sunk. Thankfully, the official rules of the game help us in this regard. Up until now, we have only used two states for giving feedback on each shot: **HIT** and **MISS**.

The official rules of the game also state that you should let your opponent know if they have successfully **SUNK** any ship, so this third style message "*You have sunk my aircraft carrier*" conveys much more information than just hit.

It tells you the length of the ship you have just hit, it tells you that you have hit all pixels of this ship, and it could, potentially, give you a new minimum size of ship you are searching for (for instance, if you have sunk all ships other than the aircraft carrier, then you know that this remaining ship is five units long and can adjust your random search according to skip the appropriate number of spaced when in hunt mode.

First thing to implement is adding some way of knowing if ship is sunk. Currently, there's no notion of a ship. Gameboard just draws squares on passed locations. We'll try to do this without completely rewriting existing classes. For that, we make a Ship class. But we add generate_separate_locations method to Random class first. That method will help us initialize ship objects.

In [1]:
import random as pyrandom
from Gameboard import *

In [2]:
class Random:
    def __init__(self):
        # possible locations list isn't affected by the methods
        self.possible_locations = []
        self.available_locations = []
        for i in Gameboard.row_labels:
            for j in Gameboard.col_labels:
                self.possible_locations.append(i + str(j))
                self.available_locations.append(i + str(j))
        
    def reset_available_locations(self):
        self.available_locations = self.possible_locations.copy()
        
    # chooses random location from available locations list and removes it if keep is False
    def choose(self, keep=False):
        choice = pyrandom.choice(self.available_locations)
        if not keep:
            self.available_locations.remove(choice)
        return choice
    
    # returns random location from possible locations list
    def choose_from_all(self):
        return pyrandom.choice(self.possible_locations)
    
    # removes passed location from available locations list 
    def make_unavailable(self, location):
        self.available_locations.remove(location)
    
    # generates random locations for ships of various length and returns them
    # WARGING: current implementation is not optimal and should be changed sometime in the future
    # NOTE: this method currently uses choose method but it resets available locations at the end
    def generate_ship_locations(self, lengths=[5, 4, 3, 3, 2]):
        flat_list = []
        for sublist in self.generate_separate_locations(lengths):
            for item in sublist:
                flat_list.append(item)
        return flat_list
    
    # returns array of ship locations, where ship locations are array of locations themselves
    # for example: [2, 3] returns [['A1', 'A2'], ['C3', 'C4', 'C5']]
    def generate_separate_locations(self, lengths=[5, 4, 3, 3, 2]):
        locations = []
        
        for length in lengths:
            orientation = pyrandom.choice(['horizontal', 'vertical'])
            
            # randomize a ship location until it fits
            while True:
                choice_pivot = self.choose(keep=True)
                choices = [choice_pivot]
                if orientation == 'horizontal':   
                    # generate all other choices (for horizontal orientation)
                    for i in range(length - 1):
                        # when orientation is horizontal, column is changed
                        choice_next = choice_pivot[0] + str(int(choice_pivot[1:]) + i + 1)
                        choices.append(choice_next)
                else:
                    # generate all other choices (for vertical orientation)
                    for i in range(length - 1):
                        # when orientation is vertical, row is changed
                        choice_next = chr(ord(choice_pivot[0]) + i + 1) + choice_pivot[1:]
                        choices.append(choice_next)
                # after generation, we must check if locations are legal
                is_legal = True
                for choice in choices:
                    if choice not in self.available_locations:
                        is_legal = False
                # if they're legal, we update available locations and break the loop
                if is_legal:
                    self.available_locations = [x for x in self.available_locations if x not in choices]
                    locations.append(choices)
                    break
        
        # reset available locations and return result
        self.reset_available_locations()
        return locations

In [3]:
# test newly added method
random = Random()
random.generate_separate_locations()

[['C7', 'D7', 'E7', 'F7', 'G7'],
 ['B7', 'B8', 'B9', 'B10'],
 ['I3', 'I4', 'I5'],
 ['A1', 'B1', 'C1'],
 ['C4', 'C5']]

In [4]:
class Ship:
    # Random class already can generate locations, so we can initialize ship after that
    def __init__(self, locations):
        self.locations = locations
        self.length = len(locations)
        self.destroyed_locations = set()
        self.is_damaged = False
        
    def get_length(self):
        return self.length
    
    def get_locations(self):
        return self.locations
    
    def destroy(self, location):
        self.is_damaged = True
        self.destroyed_locations.add(location)
        
    # returns True if at least one cell is hit
    def is_damaged():
        return self.is_damaged
        
    def is_sunk(self):
        return len(self.destroyed_locations) == self.length

In [5]:
# test Ship class
ship = Ship(random.generate_separate_locations()[0])
print(ship.get_locations())
print(ship.get_length())
print(ship.is_sunk())
for loc in ship.get_locations():
    ship.destroy(loc)
print(ship.is_sunk())

['J6', 'J7', 'J8', 'J9', 'J10']
5
False
True


Now that we will be told when a ship is sunk, we know which ships (and even more importantly what the lengths of the ships) are still active. These facts are very valuable in determining which location we search next.

Our new algorithm will calculate the most probably location to fire at next based on a superposition of all possible locations the enemy ships could be in.

At the start of every new turn, based on the ships still left in the battle, we’ll work out all possible locations that every ship could fit (horizontally or vertically).

Initially, this will be pretty much anywhere, but as more and more shots are fired, some locations become less likely, and some impossible. Every time it’s possible for a ship to be placed in over a grid location, we’ll increment a counter for that cell. The result will be a superposition of probabilities. ![PDF](https://i.imgur.com/GLmCEui.png)

In the following simple examples, we're just looking at the probabilities for the location of an aircraft carrier (length 5 units). We start in the top left corner, and try placing it horizontally. If it fits, we increment a value for each cell it lays over as a 'possible location' in which there could a ship. Then we try sliding it over one square and repeating ... and so on until we reach the end of the row. Then we move down a line and repeat. Next we repeat the exercise with the ship oriented vertically.

Sometimes the ship will fit a space, sometimes it will not. As the board becomes more and more congested (with hits, misses and sunk ships), the number of possible positions the ships can fit reduces. It's not the absolute number, however, we're looking for. We're simply looking for the most likely location for a ship to be located in based on the information we already know.

Whilst the examples below show just the probablity distributions for where an aircraft carrier could be hidden, for the full implementation we iterate through each not-yet-sunk ship adding them together to create the superposition. The algorithm selects the location with the highest count of possible ships that could be positioned through that square.

In this example, one shot has been fired, and we're looking for the aircraft carrier. It's less likely to be South (because it could not fit vertically, and so the only way it could overlap one of the cells to the South is if it layed horizontally). Similarly, it's less likely to be West. Probablities also fall off slightly towards the edges and corners as there are less ways to position the ship that covers these locations. 

![Example 1](https://i.imgur.com/xdqpaKE.png)

In this configuration we can see a white square just to the left of the top miss. It is *impossible* for the aircraft carrier to pass through this square.

![Example 2](https://i.imgur.com/PZVDR8e.png)

A ficticious board layout with lots of misses marked. Many of the locations are *impossible* to host the carrier. The darker the shading, the more possible ways that the carrier could use this square.

![Example 3](https://i.imgur.com/jpUAzzx.png)

This algorithm still has a **hunt** mode and **target** mode, though both operate essentially the same way. When in hunt mode, there are only three states to worry about: *unvisited space, misses* and *sunk ships*. Misses and Sunk ships are treated the same (obstructions that potential ships needed to be placed around). In target mode (where there is at least one hit ship that has not been sunk), then ships can, by definition, pass through this location, and so hit squares are treated as *unvisited space* squares for deciding if a ship 'could' pass through this square, and then a **heavy** score weighting is granted to possible locations that pass through a point we know already know contains a hit.